<a href="https://colab.research.google.com/github/s-i-d-d-h-a-r-t-h/NLP-Bot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json 
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

The dataset:

In [ ]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

Encoding:

In [ ]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

Tokenization and padding:

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

Creating a model:

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
Total params: 16,646
Trainable params: 16,646
Non-trainable params: 0
_________________________________________________________________


Model training:

In [ ]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
1/1 [==============================] - 1s 784ms/step - loss: 1.7895 - accuracy: 0.1538
Epoch 2/500
1/1 [==============================] - 0s 4ms/step - loss: 1.7885 - accuracy: 0.1538
Epoch 3/500
1/1 [==============================] - 0s 4ms/step - loss: 1.7875 - accuracy: 0.1538
Epoch 4/500
1/1 [==============================] - 0s 5ms/step - loss: 1.7865 - accuracy: 0.1538
Epoch 5/500
1/1 [==============================] - 0s 5ms/step - loss: 1.7855 - accuracy: 0.1538
Epoch 6/500
1/1 [==============================] - 0s 4ms/step - loss: 1.7845 - accuracy: 0.1538
Epoch 7/500
1/1 [==============================] - 0s 4ms/step - loss: 1.7836 - accuracy: 0.1538
Epoch 8/500
1/1 [==============================] - 0s 5ms/step - loss: 1.7826 - accuracy: 0.2308
Epoch 9/500
1/1 [==============================] - 0s 5ms/step - loss: 1.7816 - accuracy: 0.2308
Epoch 10/500
1/1 [==============================] - 0s 5ms/step - loss: 1.7806 - accuracy: 0.2308
Epoch 11/500
1/1 [=========

In [ ]:
model.save('chatbot.h5')

In [ ]:
def chat():
    max_len = 20
    while True:
        print("User: ", end="")
        inp = input()
        if inp.lower() == "quit" or inp.lower() == "goodbye":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]), truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print("ChatBot:", np.random.choice(i['responses']))

print("Start messaging with the bot (type quit to stop)!")
chat()

Start messaging with the bot (type quit to stop)!
User: hello
ChatBot: Hello!
User: i need help
ChatBot: I'm Tim!
User: my name is lala
ChatBot: Talk to you later
User: goodbye


This is how we can create a chatbot using Python and Deep Learning.